# Effluent Logger App

In [1]:
# we want to display the amount of effluent in each paddock
# we need to provide a start and end date
# we then need to query each of those dates for their amount of effluent and which paddocks
# we then need to sum those values
# we then display the numbers

# imports
%matplotlib widget
import sqlite3
import pandas as pd
import plotly
import ipywidgets as widgets
import numpy as np
import cufflinks as cf
import dateutil.relativedelta
import matplotlib.pyplot as plt

from datetime import datetime,timedelta

## Database stuff

In [2]:
### Database class. Allows for simple connecting and disconnecting to a database ###
class Database():
    def __init__(self, database_path):
        self.path = database_path
        self.conn = None
        self.c = None
        self.connect()
        
    def updatePath(self, path):
        self.path = path
    
    def connect(self):
        self.conn = sqlite3.connect(self.path)
        self.c = self.conn.cursor()
    
    def disconnect(self):
        self.c.close()
        self.conn.close()
    
    def query(self, query):
        return self.c.execute(query)
    
    def queryTableDF(self, query):
        # Query a table in a database and return in a DataFrame format
        return pd.read_sql_query(query, self.conn)
        
    def queryValue(self, query, row=0, column=0):
        # Return a specific value from a row/column in a queried dataframe table.
        result_df = self.queryTableDF(query)
        return result_df.iat[row,column]

### Setup the database ###
#default_database_location = 'C:\Predictive Maintenance\Maintenance.db'
default_database_location = 'D:\Current Projects\Farm\Effluent Tracker\Effluent Database.db'
db = Database(default_database_location)

### Attempt a query from the database. If there is an error indicate to user that database is disconnected ###

connected_html = widgets.HTML('<p style="color:green"><b>Connected to %s</b></p>' % default_database_location)
disconnected_html = widgets.HTML('<p style="color:red"><b> Database not connected! Check database name/location: %s</b></p>' % default_database_location)

query = 'SELECT * FROM EffluentLogs'
try:
    result = db.query(query)
    connected = True
except:
    connected = False
    
if connected:
    display(connected_html)
else:
    display(disconnected_html)

HTML(value='<p style="color:green"><b>Connected to D:\\Current Projects\\Farm\\Effluent Tracker\\Effluent Data…

# Effluent Graph

In [3]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )
    
paddocks = ['A1', 'A2', 'A3', 'A4', 'A5']
 
class EffluentTracker(widgets.HBox):
     
    def __init__(self):
        super().__init__()
        output = widgets.Output()
 
        self.end_date = datetime.today()
        self.start_date = self.end_date - dateutil.relativedelta.relativedelta(years=1)
        query=f'Select * FROM EffluentLogs WHERE Date BETWEEN "{self.start_date}" AND "{self.end_date}"'
        self.EffluentLog = db.queryTableDF(query)  
         
        with output:
            plt.close('all')
            self.fig, self.ax = plt.subplots(constrained_layout=True, figsize=(5, 3.5))
        self.bar = self.ax.bar(paddocks, self.EffluentLog[paddocks].sum().tolist())
         
        self.fig.canvas.toolbar_position = 'bottom'
        self.ax.grid(False)
 
        # define widgets
        date_start = widgets.DatePicker(
            value=pd.to_datetime(self.start_date.strftime("%Y-%m-%d")),
            description='Pick a Start Date'
        )
        date_end = widgets.DatePicker(
            value=pd.to_datetime(self.end_date.strftime("%Y-%m-%d")), 
            description='Pick an End Date'
        )
 
        controls = widgets.VBox([
            date_start, 
            date_end
        ])
        controls.layout = make_box_layout()
         
        out_box = widgets.Box([output])
        output.layout = make_box_layout()
 
        # observe stuff
        date_start.observe(self.update_start_date, 'value')
        date_end.observe(self.update_end_date, 'value')   
 
        # add to children
        self.children = [controls, output]
        
    def query_database(self, change):
        query=f'Select * FROM EffluentLogs WHERE Date BETWEEN "{self.start_date}" AND "{self.end_date}"'
        #print(query)
        self.EffluentLog = db.queryTableDF(query)  
        
    def update_graph(self, change):
        sums = self.EffluentLog[paddocks].sum().tolist()
        for bar,height in zip(self.bar, sums):
            bar.set_height(height)
        self.fig.canvas.draw()
        
    def update_start_date(self, change):
        """Draw line in plot"""
        self.start_date = change.new
        self.query_database(self)
        self.update_graph(self)
        
    def update_end_date(self, change):
        """Draw line in plot"""
        self.end_date = change.new
        self.query_database(self)
        self.update_graph(self)
 
    #def line_color(self, change):
    #    self.line.set_color(change.new)
 
    #def update_xlabel(self, change):
    #    self.ax.set_xlabel(change.new)
 
    #def update_ylabel(self, change):
    #    self.ax.set_ylabel(change.new)
    
    
         
EffluentTracker()

EffluentTracker(children=(VBox(children=(DatePicker(value=Timestamp('2020-09-17 00:00:00'), description='Pick …

# Past Effluent Logs

In [4]:
# this section alows the logging of effluent records to the database
# should check what currently exists in effluent helper in spyder
# Ideally I think this should look give a look at the most recent (perhaps 9) entries as well as a 
# blank row to enter new data
# an excel table format might be quite nice.

from ipysheet import sheet, cell, row, column, cell_range, from_dataframe

end_date = datetime.today()
start_date = end_date - dateutil.relativedelta.relativedelta(years=1)
query=f'Select * FROM EffluentLogs WHERE Date BETWEEN "{start_date}" AND "{end_date}"'
EffluentLog = db.queryTableDF(query) 
EffluentLog.index = np.arange(1,len(EffluentLog)+1)
sheet1 = from_dataframe(EffluentLog.head(10))

for k,c in enumerate(sheet1.cells):
    c.style['textAlign']='center'
    c.send_state()

sheet1

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format=None, row_end=9, row_start=0, squeeze_row=False…

# New Effluent Log

In [6]:
new_log_sheet = sheet(rows=5,columns=6, column_headers=['Date', 'A1', 'A2', 'A3', 'A4', 'A5'],
                     column_resizing=True)

new_log_sheet

button_add = widgets.Button()
widgets.VBox((new_log_sheet, button_add))

In [7]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle Code"></form>''')